[네이버 영화 평점 데이터](https://github.com/e9t/nsmc)에서 다운로드한 다음 파일들을 `torchtext`에서 로딩이 가능한 형태로 변환하자.
* ratings_train.txt
* ratings_test.txt

먼저 파일의 형태를 살펴보자.

In [2]:
!cat ratings_train.txt | head -n 10

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1
cat: 쓰기 오류: 파이프가 깨어짐


`torchtext`는 csv, json 등을 지원하는데 이 튜토리얼에서는 csv를 이용해보자. 다른 방법은 잘 모르므로 그냥 `pandas`를 이용해서...

In [4]:
import pandas as pd
columns = ['id','text','label']
train_data = pd.read_csv('data/ratings_train.txt', sep='\t', names=columns, skiprows=1)
test_data = pd.read_csv('data/ratings_test.txt', sep='\t', names=columns, skiprows=1)

In [3]:
train_data.head()

,id,text,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [5]:
train_data.to_csv('data/train_data.csv',index=False)
test_data.to_csv('data/test_data.csv',index=False)

In [7]:
!cat data/train_data.csv | head -n 5

id,text,label
9976970,아 더빙.. 진짜 짜증나네요 목소리,0
3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
10265843,너무재밓었다그래서보는것을추천한다,0
9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
cat: 쓰기 오류: 파이프가 깨어짐


csv 변환은 완료되었으므로 이제 [KoNLPy](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)의 은전한닢 tokenizer를 이용하여 잘 작동하는지 확인해보자.

In [8]:
from konlpy.tag import Mecab
mecab = Mecab()

In [9]:
print(mecab.morphs('이 밤 그날의 반딧불을 당신의 창 가까이 보낼게요'))

['이', '밤', '그날', '의', '반딧불', '을', '당신', '의', '창', '가까이', '보낼', '게요']


In [11]:
import torch
from torchtext import data

TEXT = data.Field(tokenize=mecab.morphs)
LABEL = data.LabelField(dtype = torch.float)

In [13]:
fields = {'text': ('text',TEXT), 'label': ('label',LABEL)}
# dictionary 형식은 {csv컬럼명 : (데이터 컬럼명, Field이름)}

In [14]:
train_data, test_data = data.TabularDataset.splits(
                            path = 'data',
                            train = 'train_data.csv',
                            test = 'test_data.csv',
                            format = 'csv',
                            fields = fields
)

In [15]:
vars(train_data[0])

{'text': ['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'], 'label': '0'}

완료!